In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import pandas as pd
from DateProvider import DateProvider
from datetime import datetime
from ColumnsAdder import ColumnsAdder
from MedianOfFreeBedsByKreisTableFactory import MedianOfFreeBedsByKreisTableFactory
from TimeseriesReader import readTimeseries
from IntensivstationenHtmlFileUpdater import saveLastUpdatedIntensivstationen, saveKreisOptions
from KreiseReader import readKreise
from KreisOptionsProvider import getKreisOptionsAndAlleLandkreise
from IntensiveCareBedsPersister import getAndPersistIntensiveCareBeds4AlleKreise
from Chart import createMedianOfFreeBedsByKreisChart


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')


In [3]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

26.04.2024, 06:05:05 Uhr


In [4]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = True # dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

          lastUpdated: 2023-02-03 11:49:00


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5628583fbcb3 <unknown>
#1 0x5628580ea4a7 <unknown>
#2 0x56285811ea26 <unknown>
#3 0x56285811a980 <unknown>
#4 0x5628581637b0 <unknown>
#5 0x5628581571b3 <unknown>
#6 0x56285812824a <unknown>
#7 0x562858128c1e <unknown>
#8 0x5628583bffdb <unknown>
#9 0x5628583c3f2b <unknown>
#10 0x5628583ac0f1 <unknown>
#11 0x5628583c4a92 <unknown>
#12 0x562858390faf <unknown>
#13 0x5628583eae08 <unknown>
#14 0x5628583eafe0 <unknown>
#15 0x5628583fae04 <unknown>
#16 0x7fa0f2294ac3 <unknown>


In [5]:
saveLastUpdatedIntensivstationen(
    dateProvider.getLastUpdatedDataSource(),
    toHtmlFile = "../../docs/intensivstationen.html")

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x558ea1578cb3 <unknown>
#1 0x558ea12674a7 <unknown>
#2 0x558ea129ba26 <unknown>
#3 0x558ea1297980 <unknown>
#4 0x558ea12e07b0 <unknown>
#5 0x558ea12d41b3 <unknown>
#6 0x558ea12a524a <unknown>
#7 0x558ea12a5c1e <unknown>
#8 0x558ea153cfdb <unknown>
#9 0x558ea1540f2b <unknown>
#10 0x558ea15290f1 <unknown>
#11 0x558ea1541a92 <unknown>
#12 0x558ea150dfaf <unknown>
#13 0x558ea1567e08 <unknown>
#14 0x558ea1567fe0 <unknown>
#15 0x558ea1577e04 <unknown>
#16 0x7fd5a4c94ac3 <unknown>


In [6]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

NameError: name 'needsUpdate' is not defined

In [7]:
kreisValues = sorted(timeSeries['Kreis'].drop_duplicates().values)
saveKreisOptions(
    kreisOptions = getKreisOptionsAndAlleLandkreise(kreisValues),
    toHtmlFile = "../../docs/intensivstationen.html")

NameError: name 'timeSeries' is not defined

In [8]:
getAndPersistIntensiveCareBeds4AlleKreise(
    timeSeries,
    intensivstationenDataDir = '../../docs/data/intensivstationen',
    kreisValues = kreisValues)


NameError: name 'timeSeries' is not defined

In [9]:
medianOfFreeBedsByKreisTable = MedianOfFreeBedsByKreisTableFactory(timeSeries).createMedianOfFreeBedsByKreisTable('Kreis')
medianOfFreeBedsByKreisTable.reset_index().to_json(
    '../../docs/data/intensivstationen/medianOfFreeBedsByKreisTable.json',
    orient = "records")

NameError: name 'timeSeries' is not defined

In [10]:
def getChartTitle(dateStart, dateEnd):
    def formatDate(date):
        return date.strftime("%d.%m.%Y")

    return f"Median freier Intensivbetten im Zeitraum {formatDate(dateStart)} bis {formatDate(dateEnd)}"

In [11]:
import os

createMedianOfFreeBedsByKreisChart(
    timeSeries,
    chartTitle = getChartTitle(
        dateStart=timeSeries['date'].min(),
        dateEnd=timeSeries['date'].max()),
    accessToken = os.environ['DATAWRAPPER_API_TOKEN'])

NameError: name 'timeSeries' is not defined